In [85]:
import numpy as np
!pip install import-ipynb
import import_ipynb
!pip install ipynb

In [95]:
import torch
import json
import nltk_utils
import nltk
from nltk_utils import get_ipython
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
stemmer=PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stemming(word):
    return stemmer.stem(word.lower())

def bagofwords(tokenized_sentence,all_words):
    tokenized_sentence=[stemming(w) for w in tokenized_sentence]
    bag=np.zeros(len(all_words),dtype=np.float32)
    for index,word in enumerate(all_words):
        if word in tokenized_sentence:
            bag[index]=1.0
    return bag        
    
    


In [48]:
with open("intents1.json","r") as f:
    intents=json.load(f)

In [75]:
all_words=[]
tags=[]
xy=[]
for intent in intents["intents"]:
    tags.append(intent["tag"])
    for sentence in intent["patterns"]:
        words=tokenize(sentence)
        all_words.extend(words)
        xy.append((words,intent["tag"]))

In [76]:
ignore_words=["?",".",",","!"]

In [77]:
all_words=[stemming(word) for word in all_words if word not in ignore_words]
len(all_words)

88

In [78]:
all_words=sorted(set(all_words))

In [82]:
tags=sorted(tags)
tags

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']

In [92]:
X_train=[]
y_train=[]
for (sentence,tag) in xy:
    bag=bagofwords(sentence,all_words)
    X_train.append(bag)
    label=tags.index(tag)
    y_train.append(label)
    
X_train=np.array(X_train)
y_train=np.array(y_train)

In [94]:
X_train
y_train

array([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0,
       0, 1, 1, 1])

In [103]:
class BotDataset:
    def __init__(self):
        self.num_samples=len(X_train)
        self.x_data=X_train
        self.y_data=y_train
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.num_samples

In [104]:
dataset=BotDataset()
train_loader=DataLoader(dataset=dataset,batch_size=4,shuffle=True,num_workers=2)

### Model Bulding

In [121]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,size_l1,size_l2,size_l3,num_classes):
        super().__init__()
        self.l1=nn.Linear(input_size,size_l1)
        self.l2=nn.Linear(size_l1,size_l2)
        self.l3=nn.Linear(size_l2,size_l3)
        self.l4=nn.Linear(size_l3,num_classes)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        output=self.l1(x)
        output=self.relu(output)
        output=self.l2(output)
        output=self.relu(output)
        output=self.l3(output)
        output=self.relu(output)
        output=self.l4(output)
        return output
        
            
        

In [122]:
model=NeuralNet(len(all_words),10,10,10,len(tags))

In [127]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
num_epochs=100
train_loader

In [129]:
for epoch in range(num_epochs):
    for (words,label) in train_loader:
        pred=model.forward(words)
        loss=criterion(pred,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch%10==0:
        print("epoch: {} and loss: {}".format(epoch,loss.item()))
    
    

epoch: 0 and loss: 1.9511866569519043
epoch: 10 and loss: 1.9475420713424683
epoch: 20 and loss: 1.8682262897491455
epoch: 30 and loss: 1.7405385971069336
epoch: 40 and loss: 1.4622106552124023
epoch: 50 and loss: 0.9311925172805786
epoch: 60 and loss: 1.377343773841858
epoch: 70 and loss: 0.8028870820999146
epoch: 80 and loss: 0.5381895303726196
epoch: 90 and loss: 0.2879246175289154


### Implementing Chat Bot 

In [135]:
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=10, bias=True)
  (l3): Linear(in_features=10, out_features=10, bias=True)
  (l4): Linear(in_features=10, out_features=7, bias=True)
  (relu): ReLU()
)

In [136]:
model

NeuralNet(
  (l1): Linear(in_features=54, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=10, bias=True)
  (l3): Linear(in_features=10, out_features=10, bias=True)
  (l4): Linear(in_features=10, out_features=7, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")